## 1. Introduction
<p><img src="https://assets.datacamp.com/production/project_1197/img/google_play_store.png" alt="Google Play logo"></p>
<p>Mobile apps are everywhere. They are easy to create and can be very lucrative from the business standpoint. Specifically, Android is expanding as an operating system and has captured more than 74% of the total market<sup><a href="https://www.statista.com/statistics/272698/global-market-share-held-by-mobile-operating-systems-since-2009">[1]</a></sup>. </p>
<p>The Google Play Store apps data has enormous potential to facilitate data-driven decisions and insights for businesses. In this notebook, we will analyze the Android app market by comparing ~10k apps in Google Play across different categories. We will also use the user reviews to draw a qualitative comparision between the apps.</p>
<p>The dataset you will use here was scraped from Google Play Store in September 2018 and was published on <a href="https://www.kaggle.com/lava18/google-play-store-apps">Kaggle</a>. Here are the details: <br>
<br></p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/apps.csv</b></div>
This file contains all the details of the apps on Google Play. There are 9 features that describe a given app.
<ul>
    <li><b>App:</b> Name of the app</li>
    <li><b>Category:</b> Category of the app. Some examples are: ART_AND_DESIGN, FINANCE, COMICS, BEAUTY etc.</li>
    <li><b>Rating:</b> The current average rating (out of 5) of the app on Google Play</li>
    <li><b>Reviews:</b> Number of user reviews given on the app</li>
    <li><b>Size:</b> Size of the app in MB (megabytes)</li>
    <li><b>Installs:</b> Number of times the app was downloaded from Google Play</li>
    <li><b>Type:</b> Whether the app is paid or free</li>
    <li><b>Price:</b> Price of the app in US$</li>
    <li><b>Last Updated:</b> Date on which the app was last updated on Google Play </li>

</ul>
</div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/user_reviews.csv</b></div>
This file contains a random sample of 100 <i>[most helpful first](https://www.androidpolice.com/2019/01/21/google-play-stores-redesigned-ratings-and-reviews-section-lets-you-easily-filter-by-star-rating/)</i> user reviews for each app. The text in each review has been pre-processed and passed through a sentiment analyzer.
<ul>
    <li><b>App:</b> Name of the app on which the user review was provided. Matches the `App` column of the `apps.csv` file</li>
    <li><b>Review:</b> The pre-processed user review text</li>
    <li><b>Sentiment Category:</b> Sentiment category of the user review - Positive, Negative or Neutral</li>
    <li><b>Sentiment Score:</b> Sentiment score of the user review. It lies between [-1,1]. A higher score denotes a more positive sentiment.</li>

</ul>
</div>
<p>From here on, it will be your task to explore and manipulate the data until you are able to answer the three questions described in the instructions panel.<br></p>

In [13]:
import pandas as pd

In [14]:
apps = pd.read_csv('datasets/apps.csv')
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9659 entries, 0 to 9658
Data columns (total 9 columns):
App             9659 non-null object
Category        9659 non-null object
Rating          8196 non-null float64
Reviews         9659 non-null int64
Size            8432 non-null float64
Installs        9659 non-null object
Type            9659 non-null object
Price           9659 non-null float64
Last Updated    9659 non-null object
dtypes: float64(3), int64(1), object(5)
memory usage: 679.2+ KB


In [15]:
apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,"10,000+",Free,0.0,"January 7, 2018"
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,"500,000+",Free,0.0,"January 15, 2018"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,"5,000,000+",Free,0.0,"August 1, 2018"
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,"50,000,000+",Free,0.0,"June 8, 2018"
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,"100,000+",Free,0.0,"June 20, 2018"


In [20]:
for char in [',','+'] :
    apps['Installs'] = apps['Installs'].str.replace(char,'')
apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,"January 7, 2018"
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,"January 15, 2018"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,"August 1, 2018"
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,"June 8, 2018"
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,"June 20, 2018"


In [23]:
apps['Installs'] = apps['Installs'].astype('int')
apps['Installs'].dtype

dtype('int64')

In [38]:
app_category_info=apps.groupby('Category').agg({'App':'count','Price':'mean','Rating':'mean'})
app_category_info=pd.DataFrame(app_category_info)
app_category_info.columns = ['Number of apps', 'Average price', 'Average rating']
app_category_info.head()

,Number of apps,Average price,Average rating
Category,,,
ART_AND_DESIGN,64,0.093281,4.357377
AUTO_AND_VEHICLES,85,0.158471,4.190411
BEAUTY,53,0.000000,4.278571
BOOKS_AND_REFERENCE,222,0.539505,4.344970
BUSINESS,420,0.417357,4.098479


In [52]:
user_reviews=pd.read_csv('datasets/user_reviews.csv')
user_reviews=user_reviews.groupby('App')['Sentiment Score'].mean()
user_reviews=pd.DataFrame(user_reviews)
user_reviews.head()

,Sentiment Score
App,
10 Best Foods for You,0.470733
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.392405
11st,0.181294
1800 Contacts - Lens Store,0.318145
1LINE – One Line with One Touch,0.196290


In [53]:
finance_app=apps[(apps['Category']=='FINANCE') & (apps['Price']==0)]
finance_app.head()          

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated
837,K PLUS,FINANCE,4.4,124424,NaN,10000000,Free,0.0,"June 26, 2018"
838,ING Banking,FINANCE,4.4,39041,NaN,1000000,Free,0.0,"August 3, 2018"
839,Citibanamex Movil,FINANCE,3.6,52306,42.0,5000000,Free,0.0,"July 27, 2018"
840,The postal bank,FINANCE,3.7,36718,NaN,5000000,Free,0.0,"July 16, 2018"
841,KTB Netbank,FINANCE,3.8,42644,19.0,5000000,Free,0.0,"June 28, 2018"


In [56]:
finance_app_score=finance_app.merge(user_reviews,on='App',how='inner')
finance_app_score.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated,Sentiment Score
0,Citibanamex Movil,FINANCE,3.6,52306,42.0,5000000,Free,0.0,"July 27, 2018",0.150061
1,Capitec Remote Banking,FINANCE,4.3,20672,NaN,1000000,Free,0.0,"May 8, 2018",NaN
2,Cash App,FINANCE,4.0,34428,15.0,10000000,Free,0.0,"August 2, 2018",NaN
3,Bualuang mBanking,FINANCE,4.0,48445,10.0,5000000,Free,0.0,"July 16, 2018",0.004322
4,BBVA Spain,FINANCE,4.2,36746,NaN,5000000,Free,0.0,"July 24, 2018",0.515086


In [58]:
finance_app_score.sort_values('Sentiment Score',ascending=False).ilo

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated,Sentiment Score
4,BBVA Spain,FINANCE,4.2,36746,NaN,5000000,Free,0.0,"July 24, 2018",0.515086
36,Associated Credit Union Mobile,FINANCE,4.7,3290,12.0,50000,Free,0.0,"May 30, 2018",0.388093
34,BankMobile Vibe App,FINANCE,4.3,14627,23.0,1000000,Free,0.0,"July 9, 2018",0.353455
44,A+ Mobile,FINANCE,3.9,730,6.3,10000,Free,0.0,"June 26, 2018",0.329592
29,Current debit card and app made for teens,FINANCE,4.3,685,21.0,50000,Free,0.0,"August 3, 2018",0.327258
8,BZWBK24 mobile,FINANCE,4.5,64959,39.0,1000000,Free,0.0,"June 18, 2018",0.326883
12,"Even - organize your money, get paid early",FINANCE,4.8,12304,21.0,100000,Free,0.0,"August 2, 2018",0.283929
7,Credit Karma,FINANCE,4.7,706301,NaN,10000000,Free,0.0,"July 31, 2018",0.270052
45,Fortune City - A Finance App,FINANCE,4.6,49275,91.0,500000,Free,0.0,"July 17, 2018",0.266966
19,Branch,FINANCE,4.6,69973,3.8,1000000,Free,0.0,"July 23, 2018",0.264230
